In [ ]:
!pip install transformers huggingface_hub

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="ibm-granite/granite-3.3-2b-instruct")
messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe(messages)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/787 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/207 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

Device set to use cpu


[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': 'I am an assistant designed to help answer your questions and provide information. I strive to provide concise and accurate responses in English.'}]}]

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("ibm-granite/granite-3.3-2b-instruct")
model = AutoModelForCausalLM.from_pretrained("ibm-granite/granite-3.3-2b-instruct")
messages = [
    {"role": "user", "content": "Who are you?"},
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

I am an assistant designed to help answer your questions and provide information. I strive to provide concise and accurate responses in English.<|end_of_text|>


In [10]:
!pip install gradio transformers torch sentence-transformers PyPDF2 pandas plotly

In [11]:
# Complete AI Study Assistant - Gradio Version for Google Colab
# Install required packages first: !pip install gradio transformers torch sentence-transformers PyPDF2 pandas plotly

import gradio as gr
import sqlite3
import pandas as pd
import PyPDF2
import time
import hashlib
import json
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go
from typing import List, Dict, Optional, Tuple
import io
import os
import threading

# Hugging Face models (lightweight versions for Colab)
try:
    from transformers import pipeline
    from sentence_transformers import SentenceTransformer
    import torch
    HF_AVAILABLE = True
except ImportError:
    HF_AVAILABLE = False
    print("Please install required packages: !pip install transformers torch sentence-transformers")

class ColabAI:
    """Lightweight AI class using Hugging Face models optimized for Colab"""

    def __init__(self):
        self.models = {}
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.models_loaded = False
        print(f"🔧 Device: {self.device}")

    def load_models(self):
        """Load lightweight models suitable for Colab"""
        if self.models_loaded:
            return "✅ Models already loaded!"

        try:
            print("🔄 Loading AI models (first time only)...")

            # Text generation - lightweight model
            self.models['generator'] = pipeline(
                "text-generation",
                model="distilgpt2",
                device=0 if self.device == "cuda" else -1,
                max_length=200,
                pad_token_id=50256
            )

            # Question answering
            self.models['qa'] = pipeline(
                "question-answering",
                model="distilbert-base-uncased-distilled-squad",
                device=0 if self.device == "cuda" else -1
            )

            # Summarization
            self.models['summarizer'] = pipeline(
                "summarization",
                model="sshleifer/distilbart-cnn-6-6",
                device=0 if self.device == "cuda" else -1,
                max_length=100,
                min_length=20
            )

            self.models_loaded = True
            return "✅ All AI models loaded successfully!"

        except Exception as e:
            return f"❌ Error loading models: {str(e)}"

    def generate_response(self, prompt, max_length=150):
        """Generate educational response"""
        if not self.models_loaded:
            return "⏳ Please wait, models are loading..."

        try:
            # Educational prompt formatting
            edu_prompt = f"As a helpful study assistant: {prompt}"

            result = self.models['generator'](
                edu_prompt,
                max_length=max_length,
                temperature=0.7,
                do_sample=True,
                pad_token_id=50256
            )

            response = result[0]['generated_text']
            # Clean response
            if edu_prompt in response:
                response = response.replace(edu_prompt, "").strip()

            return response if response else "I'd be happy to help! Could you be more specific about what you'd like to learn?"

        except Exception as e:
            return f"❌ Error generating response: {str(e)}"

    def answer_question(self, question, context):
        """Answer question based on context"""
        if not self.models_loaded:
            return "⏳ Please wait, models are loading..."

        try:
            # Limit context length
            if len(context) > 1000:
                context = context[:1000] + "..."

            result = self.models['qa'](question=question, context=context)

            confidence = result['score']
            answer = result['answer']

            return f"**Answer:** {answer}\n\n**Confidence:** {confidence:.2f}"

        except Exception as e:
            return f"❌ Error: {str(e)}"

    def summarize_text(self, text):
        """Summarize long text"""
        if not self.models_loaded:
            return "⏳ Please wait, models are loading..."

        try:
            if len(text) > 500:
                text = text[:500] + "..."

            result = self.models['summarizer'](text, max_length=80, min_length=20)
            return result[0]['summary_text']

        except Exception as e:
            return f"❌ Summary error: {str(e)}"

    def generate_quiz(self, content, difficulty="medium"):
        """Generate simple quiz questions"""
        if not self.models_loaded:
            return "⏳ Please wait, models are loading..."

        try:
            # Template-based quiz generation for reliability
            base_questions = {
                "easy": {
                    "question": "What is the main topic discussed in this content?",
                    "options": ["Main concept", "Secondary idea", "Background info", "Unrelated topic"],
                    "correct": 0,
                    "explanation": "The main topic is usually the primary focus of the content."
                },
                "medium": {
                    "question": "Which aspect requires the most attention when studying this topic?",
                    "options": ["Basic principles", "Advanced applications", "Historical context", "Future trends"],
                    "correct": 0,
                    "explanation": "Understanding basic principles is fundamental for mastering any topic."
                },
                "hard": {
                    "question": "How can the concepts in this content be applied practically?",
                    "options": ["Direct implementation", "Theoretical framework only", "Requires additional research", "Cannot be applied"],
                    "correct": 0,
                    "explanation": "Most educational content is designed for practical application."
                }
            }

            return base_questions.get(difficulty, base_questions["medium"])

        except Exception as e:
            return {
                "question": f"Error generating quiz: {str(e)}",
                "options": ["Try again", "Check connection", "Reload models", "Contact support"],
                "correct": 0,
                "explanation": "Please try generating the quiz again."
            }

class ColabDatabase:
    """Simple SQLite database for Colab"""

    def __init__(self):
        # Use /tmp for Colab compatibility
        self.db_path = "/tmp/study_assistant.db"
        self.init_db()

    def init_db(self):
        """Initialize database"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        cursor.execute("""
            CREATE TABLE IF NOT EXISTS chats (
                id INTEGER PRIMARY KEY,
                user_message TEXT,
                ai_response TEXT,
                timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        """)

        cursor.execute("""
            CREATE TABLE IF NOT EXISTS quizzes (
                id INTEGER PRIMARY KEY,
                question TEXT,
                user_answer TEXT,
                correct_answer TEXT,
                is_correct BOOLEAN,
                difficulty TEXT,
                timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        """)

        cursor.execute("""
            CREATE TABLE IF NOT EXISTS pdfs (
                id INTEGER PRIMARY KEY,
                filename TEXT,
                content_length INTEGER,
                upload_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        """)

        conn.commit()
        conn.close()

    def add_chat(self, user_msg, ai_response):
        """Add chat to database"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute(
            "INSERT INTO chats (user_message, ai_response) VALUES (?, ?)",
            (user_msg, ai_response)
        )
        conn.commit()
        conn.close()

    def add_quiz_result(self, question, user_answer, correct_answer, is_correct, difficulty):
        """Add quiz result"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute(
            "INSERT INTO quizzes (question, user_answer, correct_answer, is_correct, difficulty) VALUES (?, ?, ?, ?, ?)",
            (question, user_answer, correct_answer, is_correct, difficulty)
        )
        conn.commit()
        conn.close()

    def add_pdf_record(self, filename, content_length):
        """Record PDF upload"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute(
            "INSERT INTO pdfs (filename, content_length) VALUES (?, ?)",
            (filename, content_length)
        )
        conn.commit()
        conn.close()

    def get_stats(self):
        """Get basic statistics"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        cursor.execute("SELECT COUNT(*) FROM chats")
        chat_count = cursor.fetchone()[0]

        cursor.execute("SELECT COUNT(*) FROM quizzes")
        quiz_count = cursor.fetchone()[0]

        cursor.execute("SELECT COUNT(*) FROM quizzes WHERE is_correct = 1")
        correct_count = cursor.fetchone()[0]

        cursor.execute("SELECT COUNT(*) FROM pdfs")
        pdf_count = cursor.fetchone()[0]

        conn.close()

        return {
            "total_chats": chat_count,
            "total_quizzes": quiz_count,
            "correct_answers": correct_count,
            "accuracy": (correct_count / quiz_count * 100) if quiz_count > 0 else 0,
            "pdfs_processed": pdf_count
        }

    def get_recent_chats(self, limit=5):
        """Get recent chat history"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute(
            "SELECT user_message, ai_response, timestamp FROM chats ORDER BY timestamp DESC LIMIT ?",
            (limit,)
        )
        results = cursor.fetchall()
        conn.close()
        return results

# Global instances
ai_model = ColabAI() if HF_AVAILABLE else None
database = ColabDatabase()
current_pdf_content = ""
quiz_state = {"current_quiz": None, "waiting_for_answer": False}
timer_state = {"start_time": None, "duration": 25*60, "running": False}

def load_ai_models():
    """Initialize AI models"""
    if ai_model:
        return ai_model.load_models()
    return "❌ AI models not available. Please install required packages."

def chat_interface(message, history):
    """Chat interface for AI tutor"""
    if not ai_model or not ai_model.models_loaded:
        return history + [("Please load AI models first using the 'Load AI Models' button.", "")]

    if not message.strip():
        return history

    # Generate AI response
    response = ai_model.generate_response(message)

    # Save to database
    database.add_chat(message, response)

    # Update history
    history.append((message, response))

    return history

def process_pdf(pdf_file):
    """Process uploaded PDF file"""
    if pdf_file is None:
        return "❌ No PDF file uploaded.", "", ""

    try:
        # Extract text from PDF
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text_content = ""
        for page in pdf_reader.pages:
            text_content += page.extract_text() + "\n"

        global current_pdf_content
        current_pdf_content = text_content

        # Record in database
        database.add_pdf_record(pdf_file.name, len(text_content))

        # Generate summary if AI available
        summary = ""
        if ai_model and ai_model.models_loaded:
            summary = ai_model.summarize_text(text_content[:1000])
        else:
            summary = "AI summarization not available. Please load AI models first."

        stats = f"""
        📊 **PDF Statistics:**
        - **Filename:** {pdf_file.name}
        - **Pages:** {len(pdf_reader.pages)}
        - **Characters:** {len(text_content):,}
        - **Words:** {len(text_content.split()):,}
        """

        return "✅ PDF processed successfully!", summary, stats

    except Exception as e:
        return f"❌ Error processing PDF: {str(e)}", "", ""

def ask_pdf_question(question):
    """Answer questions about uploaded PDF"""
    global current_pdf_content

    if not current_pdf_content:
        return "❌ Please upload a PDF first."

    if not question.strip():
        return "❌ Please enter a question."

    if not ai_model or not ai_model.models_loaded:
        return "❌ Please load AI models first."

    return ai_model.answer_question(question, current_pdf_content)

def generate_quiz_question(topic, difficulty):
    """Generate a quiz question"""
    if not ai_model or not ai_model.models_loaded:
        return "❌ Please load AI models first.", "", "", "", "", ""

    # Use PDF content if available and topic is about PDF
    content = current_pdf_content if (current_pdf_content and "pdf" in topic.lower()) else topic

    quiz_data = ai_model.generate_quiz(content, difficulty)
    quiz_state["current_quiz"] = quiz_data
    quiz_state["waiting_for_answer"] = True

    question = quiz_data["question"]
    option_a = f"A) {quiz_data['options'][0]}"
    option_b = f"B) {quiz_data['options'][1]}"
    option_c = f"C) {quiz_data['options'][2]}"
    option_d = f"D) {quiz_data['options'][3]}"

    return question, option_a, option_b, option_c, option_d, "Select your answer above and click 'Submit Answer'"

def submit_quiz_answer(selected_option):
    """Submit quiz answer and get feedback"""
    if not quiz_state["waiting_for_answer"] or not quiz_state["current_quiz"]:
        return "❌ No active quiz question."

    quiz = quiz_state["current_quiz"]

    # Parse selected option (A, B, C, or D)
    option_map = {"A": 0, "B": 1, "C": 2, "D": 3}
    user_choice = option_map.get(selected_option.upper(), 0)
    correct_choice = quiz["correct"]

    is_correct = user_choice == correct_choice

    # Save result to database
    database.add_quiz_result(
        quiz["question"],
        quiz["options"][user_choice],
        quiz["options"][correct_choice],
        is_correct,
        "medium"  # Default difficulty
    )

    # Reset quiz state
    quiz_state["waiting_for_answer"] = False
    quiz_state["current_quiz"] = None

    # Generate feedback
    if is_correct:
        feedback = f"🎉 **Correct!** \n\n{quiz['explanation']}"
    else:
        feedback = f"""❌ **Incorrect.**

**Your Answer:** {quiz['options'][user_choice]}
**Correct Answer:** {quiz['options'][correct_choice]}

**Explanation:** {quiz['explanation']}"""

    return feedback

def start_timer(duration_minutes):
    """Start pomodoro timer"""
    global timer_state
    timer_state["duration"] = duration_minutes * 60
    timer_state["start_time"] = time.time()
    timer_state["running"] = True

    return f"⏰ Timer started for {duration_minutes} minutes!"

def check_timer():
    """Check timer status"""
    global timer_state

    if not timer_state["running"]:
        return "⏹️ Timer is not running."

    elapsed = time.time() - timer_state["start_time"]
    remaining = max(0, timer_state["duration"] - elapsed)

    if remaining <= 0:
        timer_state["running"] = False
        return "🎉 **Timer Completed!** Great job on your study session!"

    minutes = int(remaining // 60)
    seconds = int(remaining % 60)

    return f"⏰ Time remaining: {minutes:02d}:{seconds:02d}"

def stop_timer():
    """Stop the timer"""
    global timer_state
    timer_state["running"] = False
    return "⏹️ Timer stopped."

def get_analytics():
    """Generate analytics dashboard"""
    stats = database.get_stats()

    if stats["total_chats"] == 0 and stats["total_quizzes"] == 0:
        return "📊 No activity data yet. Start using the app to see analytics!"

    analytics_text = f"""
    # 📊 Learning Analytics Dashboard

    ## 📈 Overall Statistics
    - **Total Conversations:** {stats['total_chats']}
    - **Quizzes Completed:** {stats['total_quizzes']}
    - **Correct Answers:** {stats['correct_answers']}
    - **Quiz Accuracy:** {stats['accuracy']:.1f}%
    - **PDFs Processed:** {stats['pdfs_processed']}

    ## 🎯 Performance Summary
    """

    if stats['accuracy'] >= 80:
        analytics_text += "🟢 **Excellent Performance!** You're doing great!"
    elif stats['accuracy'] >= 60:
        analytics_text += "🟡 **Good Progress!** Keep up the good work!"
    else:
        analytics_text += "🔴 **Room for Improvement.** Consider reviewing topics more thoroughly."

    return analytics_text

def get_recent_activity():
    """Get recent chat activity"""
    recent_chats = database.get_recent_chats(3)

    if not recent_chats:
        return "No recent activity."

    activity_text = "## 💭 Recent Conversations:\n\n"

    for i, (user_msg, ai_response, timestamp) in enumerate(recent_chats, 1):
        activity_text += f"**{i}. Question:** {user_msg[:100]}...\n"
        activity_text += f"**Answer:** {ai_response[:100]}...\n"
        activity_text += f"**Time:** {timestamp}\n\n---\n\n"

    return activity_text

# Create Gradio Interface
def create_interface():
    """Create the main Gradio interface"""

    with gr.Blocks(theme=gr.themes.Soft(), title="🤖 AI Study Assistant") as app:

        # Header
        gr.Markdown("""
        # 🤖 AI Study Assistant
        ### Powered by Hugging Face 🤗 | Optimized for Google Colab

        A comprehensive AI-powered study companion with chat, PDF analysis, quizzes, and more!
        """)

        # Model loading section
        with gr.Row():
            load_btn = gr.Button("🚀 Load AI Models", variant="primary", size="lg")
            model_status = gr.Textbox(value="⏳ Click 'Load AI Models' to initialize",
                                    label="🤖 Model Status", interactive=False)

        load_btn.click(fn=load_ai_models, outputs=model_status)

        # Main tabs
        with gr.Tabs():

            # Dashboard Tab
            with gr.TabItem("📊 Dashboard"):
                gr.Markdown("### 🏠 Welcome to your AI Study Assistant!")

                with gr.Row():
                    with gr.Column():
                        analytics_display = gr.Markdown()
                        analytics_btn = gr.Button("🔄 Refresh Analytics")

                    with gr.Column():
                        recent_activity = gr.Markdown()
                        activity_btn = gr.Button("🔄 Refresh Activity")

                analytics_btn.click(fn=get_analytics, outputs=analytics_display)
                activity_btn.click(fn=get_recent_activity, outputs=recent_activity)

            # AI Chat Tab
            with gr.TabItem("🤖 AI Chat Tutor"):
                gr.Markdown("### 💬 Chat with your AI Study Assistant")

                chatbot = gr.Chatbot(height=400, label="AI Tutor Conversation")
                msg_input = gr.Textbox(placeholder="Ask me anything about your studies...",
                                     label="Your Message")

                with gr.Row():
                    send_btn = gr.Button("📤 Send", variant="primary")
                    clear_btn = gr.Button("🗑️ Clear Chat")

                # Suggested topics
                with gr.Row():
                    topic_btns = [
                        gr.Button("📚 Math Help"),
                        gr.Button("🔬 Science Concepts"),
                        gr.Button("📝 Study Tips"),
                        gr.Button("💡 Writing Help")
                    ]

                def send_message(message, history):
                    return chat_interface(message, history), ""

                def clear_chat():
                    return []

                def quick_topic(topic):
                    return f"Help me with {topic}", []

                send_btn.click(send_message, [msg_input, chatbot], [chatbot, msg_input])
                msg_input.submit(send_message, [msg_input, chatbot], [chatbot, msg_input])
                clear_btn.click(clear_chat, outputs=chatbot)

                # Quick topic buttons
                topic_btns[0].click(lambda: ("Help me with mathematics", []), outputs=[msg_input, chatbot])
                topic_btns[1].click(lambda: ("Explain science concepts", []), outputs=[msg_input, chatbot])
                topic_btns[2].click(lambda: ("Give me study tips", []), outputs=[msg_input, chatbot])
                topic_btns[3].click(lambda: ("Help me with writing", []), outputs=[msg_input, chatbot])

            # PDF Q&A Tab
            with gr.TabItem("📄 PDF Q&A"):
                gr.Markdown("### 📎 Upload PDF and Ask Questions")

                with gr.Row():
                    with gr.Column():
                        pdf_upload = gr.File(label="Upload PDF", file_types=[".pdf"])
                        process_status = gr.Textbox(label="Processing Status", interactive=False)

                    with gr.Column():
                        pdf_summary = gr.Textbox(label="📋 Document Summary",
                                               lines=5, interactive=False)
                        pdf_stats = gr.Markdown()

                pdf_upload.change(process_pdf, inputs=pdf_upload,
                                outputs=[process_status, pdf_summary, pdf_stats])

                # Q&A Interface
                gr.Markdown("### ❓ Ask Questions About Your PDF")
                pdf_question = gr.Textbox(placeholder="What would you like to know about this document?",
                                        label="Your Question")
                pdf_answer = gr.Textbox(label="📋 Answer", lines=5, interactive=False)
                ask_btn = gr.Button("🔍 Get Answer", variant="primary")

                ask_btn.click(ask_pdf_question, inputs=pdf_question, outputs=pdf_answer)

            # Quiz Tab
            with gr.TabItem("📝 Interactive Quiz"):
                gr.Markdown("### 🎯 Test Your Knowledge")

                with gr.Row():
                    quiz_topic = gr.Textbox(placeholder="Enter topic or use 'PDF Content'",
                                          label="📚 Quiz Topic", value="General Knowledge")
                    difficulty = gr.Dropdown(["easy", "medium", "hard"],
                                           label="🎯 Difficulty", value="medium")

                generate_btn = gr.Button("🎲 Generate Quiz Question", variant="primary")

                # Quiz display
                quiz_question = gr.Textbox(label="❓ Question", lines=3, interactive=False)

                with gr.Row():
                    with gr.Column():
                        option_a = gr.Textbox(label="Option A", interactive=False)
                        option_b = gr.Textbox(label="Option B", interactive=False)
                    with gr.Column():
                        option_c = gr.Textbox(label="Option C", interactive=False)
                        option_d = gr.Textbox(label="Option D", interactive=False)

                answer_choice = gr.Radio(["A", "B", "C", "D"], label="Your Answer")
                submit_answer_btn = gr.Button("✅ Submit Answer", variant="primary")
                quiz_feedback = gr.Markdown()

                generate_btn.click(
                    generate_quiz_question,
                    inputs=[quiz_topic, difficulty],
                    outputs=[quiz_question, option_a, option_b, option_c, option_d, quiz_feedback]
                )

                submit_answer_btn.click(
                    submit_quiz_answer,
                    inputs=answer_choice,
                    outputs=quiz_feedback
                )

            # Timer Tab
            with gr.TabItem("🍅 Pomodoro Timer"):
                gr.Markdown("### ⏰ Focus Timer for Study Sessions")

                with gr.Row():
                    timer_duration = gr.Slider(5, 60, value=25, step=5,
                                             label="⏱️ Duration (minutes)")
                    session_type = gr.Dropdown(["Study", "Break", "Review"],
                                             label="📚 Session Type", value="Study")

                with gr.Row():
                    start_timer_btn = gr.Button("▶️ Start Timer", variant="primary")
                    check_timer_btn = gr.Button("⏰ Check Timer")
                    stop_timer_btn = gr.Button("⏹️ Stop Timer")

                timer_display = gr.Textbox(label="Timer Status", interactive=False)

                start_timer_btn.click(start_timer, inputs=timer_duration, outputs=timer_display)
                check_timer_btn.click(check_timer, outputs=timer_display)
                stop_timer_btn.click(stop_timer, outputs=timer_display)

            # Analytics Tab
            with gr.TabItem("📈 Analytics"):
                gr.Markdown("### 📊 Learning Progress Analytics")

                analytics_detailed = gr.Markdown()
                refresh_analytics = gr.Button("🔄 Refresh Analytics", variant="primary")

                refresh_analytics.click(get_analytics, outputs=analytics_detailed)

        # Footer
        gr.Markdown("""
        ---
        <div style='text-align: center; color: #666;'>
            <p>🤖 AI Study Assistant | Built with Gradio 🤗</p>
            <p>Powered by Hugging Face Transformers | Optimized for Google Colab</p>
        </div>
        """)

    return app

# Launch the application
if __name__ == "__main__":
    # Create and launch the Gradio app
    app = create_interface()

    # Launch with public sharing for Colab
    app.launch(
        share=True,  # Creates public URL for Colab
        server_name="0.0.0.0",  # Allow external connections
        server_port=7860,  # Default Gradio port
        show_error=True,
        quiet=False
    )

# Instructions for Google Colab:
"""
🚀 To run in Google Colab:

1. Install packages:
   !pip install gradio transformers torch sentence-transformers PyPDF2 pandas plotly

2. Copy this entire code into a cell

3. Run the cell - it will automatically:
   - Load the Gradio interface
   - Create a public URL (like https://xxxxx.gradio.live)
   - Display the link to access your app

4. Click the generated URL to use your AI Study Assistant!

Features:
- 🤖 AI Chat Tutor (DistilGPT-2)
- 📄 PDF Upload & Q&A (DistilBERT)
- 📝 Interactive Quizzes
- 🍅 Pomodoro Timer
- 📊 Learning Analytics
- 💾 SQLite Database Storage
"""

🔧 Device: cpu


/tmp/ipython-input-979054062.py:564: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=400, label="AI Tutor Conversation")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ba0b0cf5d80d61e140.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


'\n🚀 To run in Google Colab:\n\n1. Install packages:\n   !pip install gradio transformers torch sentence-transformers PyPDF2 pandas plotly\n\n2. Copy this entire code into a cell\n\n3. Run the cell - it will automatically:\n   - Load the Gradio interface\n   - Create a public URL (like https://xxxxx.gradio.live)\n   - Display the link to access your app\n\n4. Click the generated URL to use your AI Study Assistant!\n\nFeatures:\n- 🤖 AI Chat Tutor (DistilGPT-2)\n- 📄 PDF Upload & Q&A (DistilBERT)\n- 📝 Interactive Quizzes\n- 🍅 Pomodoro Timer\n- 📊 Learning Analytics\n- 💾 SQLite Database Storage\n'